In [12]:
import pandas as pd

# Cargar el DataFrame (asegúrate de incluir las columnas necesarias)

df_final = pd.read_csv('df_final.csv')
df_final['date'] = pd.to_datetime(df_final['ts'])

    # Extraer el año
df_final['year'] = df_final['date'].dt.year

    # Eliminar zona horaria de 'date'
df_final['date'] = df_final['date'].dt.tz_localize(None)

df_final['artist'] = df_final['master_metadata_album_artist_name']

df_final['song'] = df_final['master_metadata_track_name']

df_final['min_played'] = df_final['ms_played'] / 60000

df_final['month'] = df_final['date'].dt.month


    # Definir rango de fechas
fecha_inicio = pd.Timestamp("2017-01-01")
fecha_fin = pd.Timestamp("2024-12-29")

    # Filtrar el DataFrame dentro del rango de fechas
df_filtrado = df_final[(df_final['date'] >= fecha_inicio) & (df_final['date'] <= fecha_fin)]

    # Agrupar los datos por año y mes
df_grouped = df_filtrado.groupby(['year', 'month'])['min_played'].sum().reset_index()

artist_years = df_filtrado.groupby('artist')['year'].nunique().reset_index()
artist_years = artist_years.rename(columns={'year': 'years_active'})

    # Filtrar artistas con años activos = 8
consistent_artists = artist_years[artist_years['years_active'] == 8]

    # Calcular el total de minutos reproducidos por artista
artist_playtime = df_filtrado.groupby('artist')['min_played'].sum().reset_index()
artist_playtime = artist_playtime.rename(columns={'min_played': 'total_min_played'})

    # Unir con los artistas consistentes
consistent_artists = consistent_artists.merge(artist_playtime, on='artist')

    # Ordenar por minutos reproducidos y tomar el top N



# seleccionar solo las variables a utilizar

df_final = df_final[['date', 'year', 'artist', 'song', 'ms_played', 'min_played', 'ts', 'month']]

# exportar df_final

df_final.to_csv('df_final1.csv', index=False)

C:\Users\SIMON\AppData\Local\Temp\ipykernel_8596\1145426075.py:5: DtypeWarning:

Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.



In [15]:
print(df_final.head())
print(df_final.dtypes)


                 date  year            artist  \
0 2014-07-17 17:06:21  2014    antonio orozco   
1 2014-07-17 17:06:27  2014      carlos vives   
2 2014-07-17 17:06:31  2014         rodriguez   
3 2014-07-17 17:06:34  2014  sergio contreras   
4 2014-07-17 17:06:40  2014      daddy yankee   

                                      song  ms_played  min_played  \
0                                  llegará      47461    0.791017   
1  el mar de sus ojos (feat. chocquibtown)       5061    0.084350   
2             no voy a llorar - radio edit       5479    0.091317   
3                    princesa de mi cuento       3065    0.051083   
4                         la nueva y la ex       7105    0.118417   

                     ts  month  
0  2014-07-17T17:06:21Z      7  
1  2014-07-17T17:06:27Z      7  
2  2014-07-17T17:06:31Z      7  
3  2014-07-17T17:06:34Z      7  
4  2014-07-17T17:06:40Z      7  
date          datetime64[ns]
year                   int32
artist                object
song 

In [3]:
from pandasgui import show

show(df_final)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\SIMON\AppData\Local\Temp\ipykernel_8596\864815623.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  show(df_final)
C:\Users\SIMON\AppData\Local\Temp\ipykernel_8596\864815623.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  show(df_final)
C:\Users\SIMON\AppData\Local\Temp\ipykernel_8596\864815623.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\SIMON\AppData

In [9]:
import plotly.express as px
import nbformat

# Crear el gráfico animado
fig = px.bar(
    df_grouped,
    x='min_played',
    y='artist',    color='artist',
    animation_frame='year',
    orientation='h',
    title="Evolución del Top 15 Artistas Más Consistentes (2017-2024)",
    labels={'min_played': 'Minutos Reproducidos', 'artist': 'Artista', 'year': 'Año'},
    range_x=[0, df_grouped['min_played'].max() + 5000],  # Ajusta el rango
)

# Personalización adicional
fig.update_layout(
    title={'x': 0.5, 'xanchor': 'center'},
    xaxis_title="Minutos Reproducidos",
    yaxis_title=None,
    legend_title=None,
    plot_bgcolor="black",
    paper_bgcolor="black",
    font=dict(color="white"),
)

# Mostrar el gráfico
fig.show()


ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['year', 'month', 'min_played'] but received: master_metadata_album_artist_name

In [ ]:
import plotly.io as pio

# Exportar como GIF
pio.write_image(fig, "timelapse.gif", format="gif", engine="kaleido")
